# Part 0 - Data Preparation
Before I can start I need to get data of transactions. I'm using BTC-USDT trades at Binance Perpetual Future.
The data downloaded from https://www.binance.com/en/landing/data .

The avg monthly data size *compressed* is almost 900MB. 
The avg monthly data size *uncompressed* csv is 4.7GB. 

CSV files are heavy and slow to load to pandas so the first thing that im doing is to transform it to parquet so I can reuse it later much faster and less storage needed.

The avg monthly parquet data size  is 1.3GB.


In [3]:
import os

import numpy as np
from zipfile import ZipFile 
import pandas as pd
from tqdm import tqdm as TQDM
from tqdm.notebook import tqdm

def extract_rar(file_path: str, extract_to: str, pbar: TQDM) -> str | None:
    try:
        pbar.set_description(f"Extracting: {file_path}")
        with ZipFile(file_path, 'r') as zObject: 
            zObject.extractall(extract_to)
        pbar.set_description(f"Extracted: {file_path}")
        return os.path.join(extract_to, zObject.namelist()[0])
    except Exception as e:
        print(f"Extraction failed: {file_path}, {str(e)}")
        return None

def process_csv_and_convert_to_parquet(csv_file_path: str, pbar: TQDM) -> str | None:
    try:
        pbar.set_description(f"Read: {csv_file_path}")
        df = pd.read_csv(csv_file_path)
        
        parquet_file_path = csv_file_path.replace('.csv', '.parquet')
        pbar.set_description(f"Save as {parquet_file_path}")
        df.to_parquet(parquet_file_path)
        pbar.set_description(f"Saved as {parquet_file_path}")
        
        return parquet_file_path
    except Exception as e:
        print(f"Failed to process {csv_file_path}: {str(e)}")
        return None

def iterate_and_process(folder_path: str) -> None:
    # Check if the folder path exists
    if not os.path.exists(folder_path):
        print(f"The folder '{folder_path}' does not exist.")
        return

    # Iterate over the files in the folder
    root, dirs, files = next(iter(os.walk(folder_path)))
    files = sorted(files)
    for file in (pbar := tqdm(files)):
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            # extracted_csv_path = extract_rar(file_path, folder_path, pbar)
            extracted_csv_path = file_path
            
            if extracted_csv_path and extracted_csv_path.endswith('.csv'):
                process_csv_and_convert_to_parquet(extracted_csv_path, pbar)

transaction_path = r'E:\transactions'
iterate_and_process(transaction_path)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
import humanize
import os
import numpy as np


def get_file_sizes(folder_path, file_type):
    file_sizes = []

    # Check if the folder path exists
    if not os.path.exists(folder_path):
        print(f"The folder '{folder_path}' does not exist.")
        return file_sizes

    # Iterate over the files in the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(file_type): 
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                file_sizes.append(file_size)

    print(f'Mean {file_type} file size {humanize.naturalsize(sum(file_sizes) / len(file_sizes), binary=True)}')
    print(f'Std {file_type} file size {humanize.naturalsize(np.std(file_sizes))}')
    print(f'Total {file_type} file size {humanize.naturalsize(np.sum(file_sizes))}')

# Specify the folder path
transaction_path = r'E:\transactions'
print('Zip file sizes:')
get_file_sizes(transaction_path, file_type='zip')
print('CSV file sizes:')
get_file_sizes(transaction_path, file_type='csv')
print('Parquet file sizes:')
get_file_sizes(transaction_path, file_type='parquet')

Zip file sizes:
Mean zip file size 820.3 MiB
Std zip file size 470.4 MB
Total zip file size 47.3 GB
CSV file sizes:
Mean csv file size 4.7 GiB
Std csv file size 2.7 GB
Total csv file size 277.8 GB
Parquet file sizes:
Mean parquet file size 1.3 GiB
Std parquet file size 701.0 MB
Total parquet file size 74.0 GB


In the next chapter, we work on bars generation. Its impossible to work with this size of data a local PC (mine is 64GB ram).
So my bars generators are rolling, means that I iterate over the monthly transactions and create for each of them a bars (accepting the last bar transactions to roll the last bar)  